In [1]:
from PIL import Image

In [2]:
#image to matrix
import numpy as np
import matplotlib.pyplot as plt
def ImageToMatrix(image):
    #image.show()  
    width,height = image.size
    im = image.convert("L") 
    data = im.getdata()
    data = np.matrix(data,dtype='float')/255
    new_data = np.reshape(data,(height,width))
    return new_data

In [3]:
# change the image into a 0-1 array
# threshold for the light condition
def ImageTotwovalue(image,threshold):
    #change the image into grey one
    lim=image.convert('L')  
    # set to 100 0 for black and 1 for white
    table=[]  
    # set the grey parameter
    for j in range(256):  
        if j<threshold:  
            table.append(0)  
        else:  
            table.append(1)  
  
    bim=lim.point(table,'1')  
    return bim

In [4]:
# matrix to image
def MatrixToImage(data):
    data=data*255
    new_im = Image.fromarray(data.astype(np.uint8))
    return new_im


In [5]:
# count the black points near the current point(x,y)
# sum includes the current point if it is black
# we can use this function to erase the noise points
def sumOfNearPoints(img, x, y):
    # get the color of current point 0 for black and 1 for white
    cur_color = img.getpixel((x, y))
    width = img.width
    height = img.height
    
    # if the color is white, there is no need to count the color of near points
    if cur_color == 1:  
        return 0
    
    
    # first line 
    if y == 0:
        # the upper left vertex
        if x == 0:  
            # only 3 point near it, total 4 points
            sum = cur_color + img.getpixel((x, y + 1)) + img.getpixel((x + 1, y)) + img.getpixel((x + 1, y + 1))
            # black point is 0, so the number of black points is 4 - sum
            return 4 - sum
        elif x == width - 1:  # upper right 
            sum = cur_color +img.getpixel((x, y + 1)) + img.getpixel((x - 1, y)) + img.getpixel((x - 1, y + 1))
            return 4 - sum
        else:  # upper line (not vertex), 5 near points
            sum = img.getpixel((x - 1, y)) + img.getpixel((x - 1, y + 1)) + cur_color \
                + img.getpixel((x, y + 1)) + img.getpixel((x + 1, y)) + img.getpixel((x + 1, y + 1))
            return 6 - sum
    elif y == height - 1:  # bottom line
        if x == 0:  # lower left vertex
            # 3 near points
            sum = cur_color + img.getpixel((x + 1, y)) + img.getpixel((x + 1, y - 1)) + img.getpixel((x, y - 1))
            return 4 - sum
        elif x == width - 1:  # lower right vertex
            sum = cur_color + img.getpixel((x, y - 1))+ img.getpixel((x - 1, y)) + img.getpixel((x - 1, y - 1))
            return 4 - sum
        else:  # bottom line (not vertex), 5 near points
            sum = cur_color + img.getpixel((x - 1, y)) \
                  + img.getpixel((x + 1, y)) \
                  + img.getpixel((x, y - 1)) \
                  + img.getpixel((x - 1, y - 1)) \
                  + img.getpixel((x + 1, y - 1))
            return 6 - sum
    else:  # y is not on the boarder line
        if x == 0:  # left side
            sum = img.getpixel((x, y - 1)) \
                  + cur_color \
                  + img.getpixel((x, y + 1)) \
                  + img.getpixel((x + 1, y - 1)) \
                  + img.getpixel((x + 1, y)) \
                  + img.getpixel((x + 1, y + 1))
            return 6 - sum
        elif x == width - 1:  # right side
            # print('%s,%s' % (x, y))
            sum = img.getpixel((x, y - 1)) \
                  + cur_color \
                  + img.getpixel((x, y + 1)) \
                  + img.getpixel((x - 1, y - 1)) \
                  + img.getpixel((x - 1, y)) \
                  + img.getpixel((x - 1, y + 1))
            return 6 - sum
        else:  # points that have 8 near points
            sum = img.getpixel((x - 1, y - 1)) \
                  + img.getpixel((x - 1, y)) \
                  + img.getpixel((x - 1, y + 1)) \
                  + img.getpixel((x, y - 1)) \
                  + cur_color \
                  + img.getpixel((x, y + 1)) \
                  + img.getpixel((x + 1, y - 1)) \
                  + img.getpixel((x + 1, y)) \
                  + img.getpixel((x + 1, y + 1))
            return 9 - sum

In [6]:
# this funtion is to get the board to cut the picture
def cut_get_coord_list(img_in):  
    co_list=[]
    ww=img_in.width
    hh=img_in.height
    trigger1=False   #set a triger to stop the loop
    trigger2=False
    trigger3=False
    trigger4=False
       
    for x_c in range(ww):   ## get the left x
        if trigger1==False:
            for y_c in range(hh):
                if img_in.getpixel((x_c,y_c))==0:
                    x_first=x_c
                    co_list.append(x_first)   # left
#                    print('x_1st is:'+str(x_first))
                    trigger1=True
                    break
        else:
            break
    
    for y_cf in range(hh):   ## get the upper y
        if trigger3==False:
            for x_cf in range(ww):
                if img_in.getpixel((x_cf,y_cf))==0:
                    y_first=y_cf
                    co_list.append(y_first)   #  upper
                    trigger3=True
                    break
        else:
            break   
            
    temp=0
    max0=0
    for y_cl in range(hh):
        if trigger2==False:
            x_hang_count=0
            x_hang_count1=0
            for x_cl in range(x_first,ww):
                x_hang_count=x_hang_count+img_in.getpixel((x_cl,y_cl))
            if x_hang_count > temp:
                temp = x_hang_count
                for i in range(x_first,ww):
                    x_hang_count1=x_hang_count1+img_in.getpixel((i,y_cl))
                    if x_hang_count1 == x_hang_count:
                        max0 = i
            co_list.append(max0)
            trigger2 = True
        else:
            break
                                      
    temp=0
    max0=0
    for x_cl in range(ww): 
        if trigger4==False:
            y_hang_count=0
            y_hang_count1=0
            for y_cl in range(y_first,hh):
                y_hang_count=y_hang_count+img_in.getpixel((x_cl,y_cl))
            if y_hang_count > temp:
                temp = y_hang_count
                for i in range(y_first,hh):
                    y_hang_count1=y_hang_count1+img_in.getpixel((x_cl,i))
                    if y_hang_count1 == y_hang_count:
                        max0 = i
            co_list.append(max0)
            trigger4 = True
        else:
            break
            
    return co_list

In [40]:
def ImageResult(aslFilename,backgroudFilename,thresold):
    image = Image.open(aslFilename)
    backGround = Image.open(backgroudFilename)
    image = image.resize((200,200))
    backGround = backGround.resize((200,200))
    image_result = ImageTotwovalue(image,thresold) 
    backGround_result = ImageTotwovalue(backGround,thresold)
    image_matrix=ImageToMatrix(image_result)
    backGround_matrix = ImageToMatrix(backGround_result)
    image_matrixRe = -image_matrix+backGround_matrix
    image_matrixRe = np.where(image_matrixRe==0,1,0)
    image_final = MatrixToImage(image_matrixRe)
    image_final = ImageTotwovalue(image_final,thresold)
    return image_final

In [8]:
def imageCut(image,k):
    for x in range(image.width):
        for y in range(image.height):
            if sumOfNearPoints(image,x,y) < k:
                image.putpixel((x,y),1)
    image_cut = np.array(cut_get_coord_list(image))
    image_final=image.crop(image_cut)
    return image_final


In [9]:
# get the string name of the file in the folder
import os 
def get_path(rootDir): 
    path0=[]
    for lists in os.listdir(rootDir): 
        path = os.path.join(rootDir, lists) 
        path0.append(path)
    return path0

In [64]:
# for A
Afilename = get_path("sort/A_sort/A1")
backgroundFilename = get_path("sort/nothing_sort")
for i in range(len(Afilename)): 
    for j in range(len(backgroundFilename)):
        if Afilename[i]!="sort/A_sort/A1/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":
            image = ImageResult(Afilename[i],backgroundFilename[j],100)
            k=5 #5 for A
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width
            if image.width>20:
                if image.height>20:
                    width = image.width
                    height = image.height           
                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)
                    
                    image = imageCut(image,k)
                    width = image.width
                    height = image.height  
                    
                    
                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))
            
                    im.save("result/Aresult/"+"A"+str(i)+"_Back"+str(j)+".jpg")   

KeyboardInterrupt: 

In [15]:
# for B
Bfilename = get_path("sort/B_sort/B3")
for i in range(len(Bfilename)): 
    for j in range(len(backgroundFilename)):    
        if Bfilename[i]!="sort/B_sort/B3/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":
            image = ImageResult(Bfilename[i],backgroundFilename[j],100)
            k=5 #4 for B
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width
            if image.width>25:
                if image.height>25:
                    width = image.width
                    height = image.height     

                    # clean the upper part
                    for l1 in range(width):
                        for h1 in range(10):
                            image.putpixel((l1,h1),1)
                    for l2 in range(width-40,width):
                        for h2 in range(20):
                            image.putpixel((l2,h2),1)

                    image = imageCut(image,k)
                    width = image.width
                    height = image.height     

                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)


                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))

                    im.save("result/Bresult/"+"B"+str(i)+"_Back"+str(j)+".jpg")   

In [ ]:
# for C
Cfilename = get_path("sort/C_sort/")
for i in range(len(Cfilename)):
    for j in range(len(backgroundFilename)):
        if Cfilename[i]!="sort/C_sort/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":

            image = ImageResult(Cfilename[i],backgroundFilename[j],108)
            k=5 #4 for C
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width
            if image.width>25:
                if image.height>25:
                    width = image.width
                    height = image.height

                    # clean the upper part
                    for l1 in range(width):
                        for h1 in range(37):
                            image.putpixel((l1,h1),1)


                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)

                    image = imageCut(image,k)
                    width = image.width
                    height = image.height      
                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))

                    im.save("result/Cresult/"+"C"+str(i)+"_Back"+str(j)+".jpg")   

In [ ]:
# for D
Dfilename = get_path("sort/D_sort/")
for i in range(len(Dfilename)):
    for j in range(len(backgroundFilename)):
        if Dfilename[i]!="sort/D_sort/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store" :
            image = ImageResult(Dfilename[i],"sort/nothing/nothing11.jpg",102)
            k=5
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width
            if image.width>25:
                if image.height>25:
                    width = image.width
                    height = image.height

                    # clean the upper part
                    for l1 in range(width):
                        for h1 in range(int(height/8)):
                            image.putpixel((l1,h1),1)

                    # clean the left bottom part
                    for l2 in range(int(width/2.2),width):
                        for h2 in range(50):
                            image.putpixel((l2,h2),1)

                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)

                    image = imageCut(image,k)
                    width = image.width
                    height = image.height       
                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))

                    im.save("result/Dresult/"+"D"+str(i)+"_Back"+str(j)+".jpg")   

In [ ]:
# for E
Efilename = get_path("sort/E_sort/")
for i in range(len(Efilename)):  
    for j in range(len(backgroundFilename)):
        if Efilename[i]!="sort/E_sort/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":
            image = ImageResult(Efilename[i],"sort/nothing/nothing11.jpg",104)
            k=5
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width
            if image.width>25:
                if image.height>25:
                    width = image.width
                    height = image.height

                    # clean the upper part
                    for l1 in range(width):
                        for h1 in range(int(height/8)+7):
                            image.putpixel((l1,h1),1)

                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)
                    image = imageCut(image,k)
                    width = image.width
                    height = image.height  


                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))

                    im.save("result/Eresult/"+"E"+str(i)+"_Back"+str(j)+".jpg")   

In [ ]:
# for F
Ffilename = get_path("sort/F_sort/")
for i in range(len(Ffilename)):
    for j in range(len(backgroundFilename)):
        if Ffilename[i]!="sort/F_sort/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":
            image = ImageResult(Ffilename[i],"sort/nothing/nothing10.jpg",101)
            k=5
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width
            if image.width>25:
                if image.height>25:
                    width = image.width
                    height = image.height           
                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)

                    image = imageCut(image,k)
                    width = image.width
                    height = image.height  

                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))

                    im.save("result/Fresult/"+"F"+str(i)+"_Back"+str(j)+".jpg")   

In [ ]:
# for G
Gfilename = get_path("sort/G_sort/")
for i in range(len(Gfilename)):
    for j in range(len(backgroundFilename)):
        if Gfilename[i]!="sort/G_sort/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":
            image = ImageResult(Gfilename[i],"sort/nothing/nothing188.jpg",50)
            k=5
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width
            if image.width>25:
                if image.height>25:
                    width = image.width
                    height = image.height


                     # clean the upper part
                    for l1 in range(width):
                        for h1 in range(20):
                            image.putpixel((l1,h1),1)

                    # clean the left bottom part
                    for l2 in range(int(width/3)):
                        for h2 in range(height-30,height):
                            image.putpixel((l2,h2),1)



                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)

                    image = imageCut(image,k)
                    width = image.width
                    height = image.height      
                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))

                    im.save("result/Gresult/"+"G"+str(i)+"_Back"+str(j)+".jpg")   

In [16]:
# for H
Hfilename = get_path("sort/H_sort/")
for i in range(len(Hfilename)):
    for j in range(len(backgroundFilename)):
        if Hfilename[i]!="sort/H_sort/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":
            image = ImageResult(Hfilename[i],"sort/nothing/nothing219.jpg",85)
            k=5
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),110)
            #select better output and the turn it into same height and width
            if image.width>25:
                if image.height>25:
                    width = image.width
                    height = image.height

                    # clean the upper part
                    for l1 in range(width):
                        for h1 in range(16):
                            image.putpixel((l1,h1),1)

                    for l3 in range(width-65,width):
                        for h3 in range(40):
                            image.putpixel((l3,h3),1)

                    # clean the left bottom part
                    for l2 in range(int(width/2)):
                        for h2 in range(height-70,height):
                            image.putpixel((l2,h2),1)

                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)
                    image = imageCut(image,k)
                    width = image.width
                    height = image.height                     

                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))


                    im.save("result/Hresult/"+"H"+str(i)+"_Back"+str(j)+".jpg")   

In [ ]:
# for i
Ifilename = get_path("sort/I_sort/")
for i in range(len(Ifilename)):
    for j in range(len(backgroundFilename)):
        if Ifilename[i]!="sort/I_sort/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":
            image = ImageResult(Ifilename[i],"asl-alphabet/asl_alphabet_train/nothing/nothing891.jpg",105)
            k=5
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width
            if image.width>25:
                if image.height>25:
                    width = image.width
                    height = image.height


                     # clean the upper part
                    for l1 in range(width):
                        for h1 in range(30):
                            image.putpixel((l1,h1),1)

                    # clean the left bottom part
                    for l2 in range(70):
                        for h2 in range(height-70,height):
                            image.putpixel((l2,h2),1)
                    image = imageCut(image,k)
                    width = image.width
                    height = image.height                         


                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)


                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))

                    im.save("result/Iresult/"+"I"+str(i)+"_Back"+str(j)+".jpg")   

In [ ]:
# for J
Jfilename = get_path("sort/J_sort/")
for i in range(len(Ifilename)):
    for j in range(len(backgroundFilename)):
        if Jfilename[i]!="sort/J_sort/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":
            image = ImageResult(Jfilename[i],"sort/nothing/nothing11.jpg",90)
            k=5
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width
            if image.width>25:
                if image.height>25:
                    width = image.width
                    height = image.height


                     # clean the upper part
                    for l1 in range(width):
                        for h1 in range(25):
                            image.putpixel((l1,h1),1)

                    # clean the left bottom part
                    for l2 in range(55):
                        for h2 in range(height-60,height):
                            image.putpixel((l2,h2),1)

                    # clean the upper right part
                    for l3 in range(width-70,width):
                        for h3 in range(35):
                            image.putpixel((l3,h3),1)

                    image = imageCut(image,k)
                    width = image.width
                    height = image.height                         

                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)


                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))

                    im.save("result/Jresult/"+"J"+str(i)+"_Back"+str(j)+".jpg")   

In [ ]:
# for k
Kfilename = get_path("sort/K_sort/")
for i in range(len(Kfilename)):
    for j in range(len(backgroundFilename)):
        if Kfilename[i]!="sort/K_sort/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":
            image = ImageResult(Kfilename[i],"sort/nothing/nothing930.jpg",90)
            k=5
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width
            if image.width>25:
                if image.height>25:
                    width = image.width
                    height = image.height


                     # clean the upper part
                    for l1 in range(width):
                        for h1 in range(30):
                            image.putpixel((l1,h1),1)

                    # clean the left bottom part
                    for l2 in range(20):
                        for h2 in range(height-35,height):
                            image.putpixel((l2,h2),1)

                    for l3 in range(10):
                        for h3 in range(height-70,height):
                            image.putpixel((l3,h3),1)
                    image = imageCut(image,k)
                    width = image.width
                    height = image.height                        


                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)


                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))

                    im.save("result/Kresult/"+"K"+str(i)+"_Back"+str(j)+".jpg")   

In [ ]:
# for L
Lfilename = get_path("sort/L_sort/")
for i in range(len(Lfilename)):
    for j in range(len(backgroundFilename)):
        if Lfilename[i]!="sort/L_sort/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":
            image = ImageResult(Lfilename[i],"sort/nothing/nothing142.jpg",90)
            k=5
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width
            if image.width>25:
                if image.height>25:
                    width = image.width
                    height = image.height


                     # clean the upper part
                    for l1 in range(width):
                        for h1 in range(30):
                            image.putpixel((l1,h1),1)

                    # clean the left bottom part
                    for l2 in range(20):
                        for h2 in range(height-20,height):
                            image.putpixel((l2,h2),1)

                    image = imageCut(image,k)
                    width = image.width
                    height = image.height                                                
                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)


                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))

                    im.save("result/Lresult/"+"L"+str(i)+"_Back"+str(j)+".jpg")   

In [19]:
# for M
Mfilename = get_path("sort/M_sort/")
for i in range(len(Mfilename)):
    for j in range(len(backgroundFilename)):
        if Mfilename[i]!="sort/M_sort/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":
            image = ImageResult(Mfilename[i],"sort/nothing/nothing142.jpg",65)
            k=5
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width
            if image.width>25:
                if image.height>25:
                    width = image.width
                    height = image.height


                     # clean the upper part
                    for l1 in range(width):
                        for h1 in range(30):
                            image.putpixel((l1,h1),1)

                    # clean the left bottom part
                    for l2 in range(20):
                        for h2 in range(height-20,height):
                            image.putpixel((l2,h2),1)
                    image = imageCut(image,k)
                    width = image.width
                    height = image.height                         

                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)


                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))

                    im.save("result/Mresult/"+"M"+str(i)+"_Back"+str(j)+".jpg")   

# I notice that the black image of M and N is very similar, i think this will cause a big issue when recognising these two alphabets.

In [20]:
# for N
Nfilename = get_path("sort/N_sort/")
for i in range(len(Nfilename)):
    for j in range(len(backgroundFilename)):
        if Nfilename[i]!="sort/N_sort/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":      
                                                                                               # i set this number
                                                                                               # to be low so that 
                                                                                               # we can see the hand shape
                                                                                               # in the shadow
            image = ImageResult(Nfilename[i],"sort/nothing/nothing77.jpg",65)
            k=5
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width
            if image.width>25:
                if image.height>25:
                    width = image.width
                    height = image.height


                     # clean the upper part
                    for l1 in range(width):
                        for h1 in range(25):
                            image.putpixel((l1,h1),1)

                    for l3 in range(int(2.5*width/4),width):
                        for h3 in range(35):
                            image.putpixel((l3,h3),1)

                    # clean the left bottom part
                    for l2 in range(20):
                        for h2 in range(height-35,height):
                            image.putpixel((l2,h2),1)
                    image = imageCut(image,k)
                    width = image.width
                    height = image.height                         

                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)


                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))

                    im.save("result/Nresult/"+"N"+str(i)+"_Back"+str(j)+".jpg")   

In [ ]:
# for O
Ofilename = get_path("sort/O_sort/")
for i in range(len(Ofilename)):
    for j in range(len(backgroundFilename)):
        if Ofilename[i]!="sort/O_sort/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":
            image = ImageResult(Ofilename[i],"sort/nothing/nothing157.jpg",100)
            k=5
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width
            if image.width>25:
                if image.height>25:
                    width = image.width
                    height = image.height


                     # clean the upper part
                    for l1 in range(width):
                        for h1 in range(25):
                            image.putpixel((l1,h1),1)

                    for l3 in range(int(3*width/4),width):
                        for h3 in range(35):
                            image.putpixel((l3,h3),1)



                    # clean the left bottom part
                    for l2 in range(25):
                        for h2 in range(height-65,height):
                            image.putpixel((l2,h2),1)

                    for l4 in range(40):
                        for h4 in range(height-40,height):
                            image.putpixel((l4,h4),1)

                    image = imageCut(image,k)
                    width = image.width
                    height = image.height                                                
                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)


                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))

                    im.save("result/Oresult/"+"O"+str(i)+"_Back"+str(j)+".jpg")   

In [ ]:
# for P
Pfilename = get_path("sort/P_sort/")
for i in range(len(Pfilename)):
    for j in range(len(backgroundFilename)):
        if Pfilename[i]!="sort/P_sort/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":
            image = ImageResult(Pfilename[i],"sort/nothing/nothing31.jpg",90)
            k=5
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width
            if image.width>25:
                if image.height>25:
                    width = image.width
                    height = image.height


                     # clean the upper part
                    for l1 in range(width):
                        for h1 in range(25):
                            image.putpixel((l1,h1),1)

                    for l3 in range(int(width/2),width):
                        for h3 in range(30):
                            image.putpixel((l3,h3),1)

                    for l4 in range(int(3*width/4),width):
                        for h4 in range(40):
                            image.putpixel((l4,h4),1)


                    # clean the left bottom part
                    for l2 in range(30):
                        for h2 in range(height-50,height):
                            image.putpixel((l2,h2),1)

                    image = imageCut(image,k)
                    width = image.width
                    height = image.height                                                
                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)


                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))

                    im.save("result/Presult/"+"P"+str(i)+"_Back"+str(j)+".jpg")   

In [ ]:
# for Q
Qfilename = get_path("sort/Q_sort/")
for i in range(len(Qfilename)):
    for j in range(len(backgroundFilename)):
        if Qfilename[i]!="sort/Q_sort/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":
            image = ImageResult(Qfilename[i],"sort/nothing/nothing1723.jpg",85)
            k=5
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width
            if image.width>25:
                if image.height>25:
                    width = image.width
                    height = image.height


                     # clean the upper part
                    for l1 in range(width):
                        for h1 in range(25):
                            image.putpixel((l1,h1),1)
                    # clean the left bottom part
                    for l2 in range(25):
                        for h2 in range(height-60,height):
                            image.putpixel((l2,h2),1)
                    image = imageCut(image,k)
                    width = image.width
                    height = image.height                                    
                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)


                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))

                    im.save("result/Qresult/"+"Q"+str(i)+"_Back"+str(j)+".jpg")   

In [ ]:
# for R
Rfilename = get_path("sort/R_sort/")
for i in range(len(Rfilename)):
    for j in range(len(backgroundFilename)):
        if Rfilename[i]!="sort/R_sort/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":
            image = ImageResult(Rfilename[i],"sort/nothing/nothing1723.jpg",90)
            k=5
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width
            if image.width>25:
                if image.height>25:
                    width = image.width
                    height = image.height

                     # clean the upper part
                    for l1 in range(width):
                        for h1 in range(20):
                            image.putpixel((l1,h1),1)

                    for l3 in range(int(3.5*width/4),width):
                        for h3 in range(40):
                            image.putpixel((l3,h3),1)
                    # clean the left bottom part
                    for l2 in range(24):
                        for h2 in range(height-55,height):
                            image.putpixel((l2,h2),1)
                    image = imageCut(image,k)
                    width = image.width
                    height = image.height                                    
                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)


                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))

                    im.save("result/Rresult/"+"R"+str(i)+"_Back"+str(j)+".jpg")   

In [ ]:
# for S
Sfilename = get_path("sort/S_sort/")
for i in range(len(Sfilename)):
    for j in range(len(backgroundFilename)):
        if Sfilename[i]!="sort/S_sort/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":
            image = ImageResult(Sfilename[i],"sort/nothing/nothing173.jpg",95)
            k=5
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width
            if image.width>25:
                if image.height>25:
                    width = image.width
                    height = image.height

                     # clean the upper part
                    for l1 in range(width):
                        for h1 in range(25):
                            image.putpixel((l1,h1),1)


                    # clean the left bottom part
                    for l2 in range(40):
                        for h2 in range(height-55,height):
                            image.putpixel((l2,h2),1)
                    image = imageCut(image,k)
                    width = image.width
                    height = image.height                                    
                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)


                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))

                    im.save("result/Sresult/"+"S"+str(i)+"_Back"+str(j)+".jpg")   

In [ ]:
# for T
Tfilename = get_path("sort/T_sort/")
for i in range(len(Tfilename)):
    for j in range(len(backgroundFilename)):
        if Tfilename[i]!="sort/T_sort/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":
            image = ImageResult(Tfilename[i],"sort/nothing/nothing1723.jpg",100)
            k=5
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width
            if image.width>25:
                if image.height>25:
                    width = image.width
                    height = image.height

                     # clean the upper part
                    for l1 in range(width):
                        for h1 in range(25):
                            image.putpixel((l1,h1),1)


                    # clean the left bottom part
                    for l2 in range(30):
                        for h2 in range(height):
                            image.putpixel((l2,h2),1)
                    image = imageCut(image,k)
                    width = image.width
                    height = image.height                                    
                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)


                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))

                    im.save("result/Tresult/"+"T"+str(i)+"_Back"+str(j)+".jpg")   

In [ ]:
# for U
Ufilename = get_path("sort/U_sort/")
for i in range(len(Ufilename)):
    for j in range(len(backgroundFilename)):
        if Ufilename[i]!="sort/U_sort/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":
            image = ImageResult(Ufilename[i],"sort/nothing/nothing87.jpg",97)
            k=5
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width
            if image.width>25:
                if image.height>25:
                    width = image.width
                    height = image.height

                     # clean the upper part
                    for l1 in range(width):
                        for h1 in range(25):
                            image.putpixel((l1,h1),1)

                    for l3 in range(width-90,width):
                        for h3 in range(40):
                            image.putpixel((l3,h3),1)



                    # clean the left bottom part
                    for l2 in range(40):
                        for h2 in range(height-70,height):
                            image.putpixel((l2,h2),1)
                    image = imageCut(image,k)
                    width = image.width
                    height = image.height                                    
                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)


                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))

                    im.save("result/Uresult/"+"U"+str(i)+"_Back"+str(j)+".jpg")   

In [ ]:
# for V
Vfilename = get_path("sort/V_sort/")
for i in range(len(Vfilename)):
    for j in range(len(backgroundFilename)):
        if Vfilename[i]!="sort/V_sort/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":
            image = ImageResult(Vfilename[i],"sort/nothing/nothing1723.jpg",65)
            k=5
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width
            if image.width>25:
                if image.height>25:
                    width = image.width
                    height = image.height

                     # clean the upper part
                    for l1 in range(width):
                        for h1 in range(20):
                            image.putpixel((l1,h1),1)



                    # clean the left bottom part
                    for l2 in range(30):
                        for h2 in range(height-30,height):
                            image.putpixel((l2,h2),1)
                    image = imageCut(image,k)
                    width = image.width
                    height = image.height                                    
                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)


                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))

                    im.save("result/Vresult/"+"V"+str(i)+"_Back"+str(j)+".jpg")   

In [ ]:
# for W
Wfilename = get_path("sort/W_sort/")
for i in range(len(Wfilename)):
    for j in range(len(backgroundFilename)):
        if Wfilename[i]!="sort/W_sort/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":
            image = ImageResult(Wfilename[i],"sort/nothing/nothing173.jpg",90)
            k=5
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width
            if image.width>25:
                if image.height>25:
                    width = image.width
                    height = image.height

                     # clean the upper part
                    for l1 in range(width):
                        for h1 in range(15):
                            image.putpixel((l1,h1),1)

                    for l3 in range(width-80,width):
                        for h3 in range(40):
                            image.putpixel((l3,h3),1)



                    # clean the left bottom part
                    for l2 in range(30):
                        for h2 in range(height-60,height):
                            image.putpixel((l2,h2),1)
                    image = imageCut(image,k)
                    width = image.width
                    height = image.height                                    
                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)


                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))

                    im.save("result/Wresult/"+"W"+str(i)+"_Back"+str(j)+".jpg")   

In [ ]:
### for X
Xfilename = get_path("sort/X_sort/")
for i in range(len(Xfilename)):
    for j in range(len(backgroundFilename)):
        if Xfilename[i]!="sort/X_sort/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":
            image = ImageResult(Xfilename[i],"sort/nothing/nothing173.jpg",88)
            k=5
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width
            if image.width>25:
                if image.height>25:
                    width = image.width
                    height = image.height

                     # clean the upper part
                    for l1 in range(width):
                        for h1 in range(23):
                            image.putpixel((l1,h1),1)



                    # clean the left bottom part
                    for l2 in range(60):
                        for h2 in range(height-70,height):
                            image.putpixel((l2,h2),1)
                    image = imageCut(image,k)
                    width = image.width
                    height = image.height                                    
                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)


                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))

                    im.save("result/Xresult/"+"X"+str(i)+"_Back"+str(j)+".jpg")   

In [ ]:
### for Y
Yfilename = get_path("sort/Y_sort/")
for i in range(len(Yfilename)):
    for j in range(len(backgroundFilename)):
        if Yfilename[i]!="sort/Y_sort/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":
            image = ImageResult(Yfilename[i],"sort/nothing/nothing173.jpg",88)
            k=5
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width
            if image.width>25:
                if image.height>25:
                    width = image.width
                    height = image.height

                     # clean the upper part
                    for l1 in range(width):
                        for h1 in range(20):
                            image.putpixel((l1,h1),1)



                    # clean the left bottom part
                    for l2 in range(40):
                        for h2 in range(height-40,height):
                            image.putpixel((l2,h2),1)

                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)

                    image = imageCut(image,k)
                    width = image.width
                    height = image.height                     
                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))

                    im.save("result/Yresult/"+"Y"+str(i)+"_Back"+str(j)+".jpg")   

In [ ]:
### for Z
Zfilename = get_path("sort/Z_sort/")
for i in range(len(Zfilename)):
    for j in range(len(backgroundFilename)):
        if Zfilename[i]!="sort/Z_sort/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":
            image = ImageResult(Zfilename[i],"sort/nothing/nothing173.jpg",80)
            k=5
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width
            if image.width>25:
                if image.height>25:
                    width = image.width
                    height = image.height

                     # clean the upper part
                    for l1 in range(width):
                        for h1 in range(25):
                            image.putpixel((l1,h1),1)

                    # clean the left bottom part
                    for l2 in range(35):
                        for h2 in range(height-20,height):
                            image.putpixel((l2,h2),1)

                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)
                    image = imageCut(image,k)
                    width = image.width
                    height = image.height                     

                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))

                    im.save("result/Zresult/"+"Z"+str(i)+"_Back"+str(j)+".jpg")   

In [ ]:
### for space
spacefilename = get_path("sort/space_sort/")
for i in range(len(spacefilename)):
    for j in range(len(backgroundFilename)):
        if spacefilename[i]!="sort/space_sort/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":
            image = ImageResult(spacefilename[i],"sort/nothing/nothing87.jpg",95)
            k=5
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width
            if image.width>25:
                if image.height>25:
                    width = image.width
                    height = image.height

                     # clean the upper part
                    for l1 in range(width):
                        for h1 in range(35):
                            image.putpixel((l1,h1),1)

                    # clean the left bottom part
                    for l2 in range(38):
                        for h2 in range(height-40,height):
                            image.putpixel((l2,h2),1)

                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)
                    image = imageCut(image,k)
                    width = image.width
                    height = image.height                     

                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))

                    im.save("result/spaceresult/"+"space"+str(i)+"_Back"+str(j)+".jpg")   

In [ ]:
### for del
delfilename = get_path("sort/del_sort/")
for i in range(len(delfilename)):
    for j in range(len(backgroundFilename)):
        if delfilename[i]!="sort/del_sort/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":
            image = ImageResult(delfilename[i],"sort/nothing/nothing87.jpg",80)
            k=5
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width
            if image.width>25:
                if image.height>25:
                    width = image.width
                    height = image.height

                     # clean the upper part
                    for l1 in range(width):
                        for h1 in range(35):
                            image.putpixel((l1,h1),1)

                    # clean the left bottom part
                    for l2 in range(20):
                        for h2 in range(height-30,height):
                            image.putpixel((l2,h2),1)

                    if width % 2==0:
                        startIndex = 100-int(width/2)                
                    else:
                        startIndex = 100-int((width-1)/2)
                    image = imageCut(image,k)
                    width = image.width
                    height = image.height                     

                    for j in range(height):
                        for h in range(width):
                            im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))

                    im.save("result/delresult/"+"del"+str(i)+"_Back"+str(j)+".jpg")   

In [59]:
# for Test
Afilename = get_path("test")
for i in range(len(Afilename)):
    for j in range(len(backgroundFilename)):
        if Afilename[i]!="test/.DS_Store" and \
        backgroundFilename[j] != "sort/nothing_sort/.DS_Store":
            image = ImageResult(Afilename[i],"nothing.jpg",150)
            k=4 #5 for A
            #first cut
            image = imageCut(image,k)
            im = ImageTotwovalue(Image.new("RGB",(200,200),"white"),100)
            #select better output and the turn it into same height and width

            width = image.width
            height = image.height 

                    # clean the upper part
            for l1 in range(width):
                for h1 in range(25):
                    image.putpixel((l1,h1),1)
            image = imageCut(image,k)    
            width = image.width
            height = image.height 
            if width % 2==0:
                startIndex = 100-int(width/2)                
            else:
                startIndex = 100-int((width-1)/2)
            image = imageCut(image,k)
            width = image.width
            height = image.height                     

            for j in range(height):
                for h in range(width):
                    im.putpixel((h+startIndex,200-height-1+j),image.getpixel((h,j)))

            im.save("test0/"+"test"+str(i)+".jpg")   

In [ ]:
# code for adjust
# for i in range(10):
#    image = ImageResult("sort/del_sort/del2607.jpg",\
#                    "sort/nothing/nothing"+str(i+87)+".jpg",80)
#    a=imageCut(image,5)
#    a.show()
